In [3]:
binsize = 10
input_dir = '/Users/tobias/Downloads/Mipas/'
output_dir = '.'

to_exclude = ['altitude', 'ExcludeVar2']

# Find all the input files in the input directory
input_files = glob.glob(os.path.join(input_dir, 'MIPAS-E_IMK.??????.V8R_CH4_*61.nc'))

for input_file in input_files:
    print('processing ' + input_file)
    
    # Create the output file names for the corrected and binned data
    output_file_corr = os.path.join(output_dir, os.path.basename(input_file).replace('.nc', '_corr.nc'))
    output_file_bin = os.path.join(output_dir, os.path.basename(input_file).replace('.nc', '_bin' + str(binsize) + '_1M.nc'))

    to_exclude = ['altitude', 'ExcludeVar2']

    with nc.Dataset(input_file, 'r') as src, nc.Dataset(output_file_corr, "w") as dst:
        # Copy global attributes all at once via dictionary
        dst.setncatts(src.__dict__)
        
        # Copy dimensions
        for name, dimension in src.dimensions.items():
            dst.createDimension(name, len(dimension) if not dimension.isunlimited() else None)
        
        # Copy all file data except for the excluded variables
        for name, variable in src.variables.items():
            if name == 'altitude':
                # Rename the altitude variable to height
                x = dst.createVariable('height', variable.dtype, variable.dimensions)
                dst['height'][:] = src[name][:]
                dst['height'].setncatts(src[name].__dict__)
                x = dst.createVariable('altitude', variable.dtype, ('altitude',))
                dst[name][:] = np.nanmean(src[name][:], axis=1)
                dst['height'].setncatts(src['altitude'].__dict__)
            if name not in to_exclude:
                x = dst.createVariable(name, variable.datatype, variable.dimensions)
                dst[name][:] = src[name][:]
                dst[name].setncatts(src[name].__dict__)

        ds = xr.open_dataset(xr.backends.NetCDF4DataStore(dst))
        ds.load()
        
    # Now bin the data and write a monthly mean of the binned data.

    # Resample time to monthly intervals
    monthly_time = ds.time.resample(time="1M").mean().values

    # Define the bin edges for latitude and longitude
    nblat = int((90 - (-90)) / binsize)+1
    nblon = int((180 - (-180)) / binsize)+1
    lat_bins = np.linspace(-90, 90, nblat)
    lon_bins = np.linspace(-180, 180, nblon)

    # Bin the latitude and longitude values
    lat_indices = np.digitize(ds.latitude.values, lat_bins)
    lon_indices = np.digitize(ds.longitude.values, lon_bins)

    # Initialize an empty array to store the binned CH4 values
    n_altitudes = ds.height.shape[0]
    n_lats = len(lat_bins) - 1
    n_lons = len(lon_bins) - 1
    n_time = 1
    ch4_binned = np.empty((n_time, n_altitudes, n_lats, n_lons))
    ch4_binned[:] = np.nan

    # Loop over altitudes and fill in the binned CH4 values
    for alt_idx in range(n_altitudes):
        ch4_alt = ds.target.values[alt_idx]
        for lat_idx in range(n_lats):
            for lon_idx in range(n_lons):
                mask = (lat_indices == lat_idx+1) & (lon_indices == lon_idx+1)
                if np.any(mask):
                    ch4_binned[0, alt_idx, lat_idx, lon_idx] = np.nanmean(ch4_alt[mask])
                else:
                    ch4_binned[0, alt_idx, lat_idx, lon_idx] = np.nan

    # Create a new xarray dataset with the binned CH4 values
    ds_binned = xr.Dataset(
        {"ch4": (["time", "altitude", "latitude", "longitude"], ch4_binned)},
        coords={
            "time": monthly_time,
            "altitude": ds.altitude.values,
            "latitude": lat_bins[:-1] + np.diff(lat_bins)/2,
            "longitude": lon_bins[:-1] + np.diff(lon_bins)/2,
        },
        attrs=ds.attrs,
    )
    ds_binned["ch4"].attrs = ds["target"].attrs

    # Write the monthly average of the binned (lat,lon) data into a netcdf file: 
    ds_binned.to_netcdf(output_file_bin)


processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200601.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200702.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200811.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200804.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200907.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200912.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200609.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200710.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200705.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200606.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200803.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.201201.V8R_CH4_261.nc
processing /Users/tobias/Downloads/Mipas/MIPAS-E_IMK.200908.V8R_CH4_261.nc
processing /Users/tobias/